<a href="https://colab.research.google.com/github/shreya82004/Development-of-Interactive-Cyber-Threat-Visualization-Dashboard-/blob/main/SQL_TASK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

now for same dataset i want to do data analysis using sql in google collb

In [ ]:
import pandas as pd

df = pd.read_csv("cyber_threat_dataset.csv")
df.head()


,incident_id,timestamp,src_ip,dest_ip,src_country,dest_country,attack_type,severity,protocol,src_port,dest_port,bytes_sent,bytes_received,packets,duration_sec,response_time_ms,action_taken,attack_status,is_malicious
0,INC001,2025-01-01 10:12:45,185.220.101.5,10.0.0.12,Russia,India,DDoS,High,TCP,44321,80,950000,120000,1200,35,180,Blocked IP,Mitigated,1
1,INC002,2025-01-01 10:25:10,192.168.1.23,10.0.0.15,India,India,Port Scan,Medium,TCP,51514,22,12000,5000,150,12,90,Logged,Detected,1
2,INC003,2025-01-01 11:02:33,45.83.64.19,10.0.0.20,Germany,India,Brute Force,High,TCP,39822,22,48000,14000,420,18,110,Account Locked,Mitigated,1
3,INC004,2025-01-01 11:40:55,172.16.0.8,10.0.0.25,India,India,Normal Traffic,Low,UDP,54678,53,8000,9200,60,6,25,Allowed,Benign,0
4,INC005,2025-01-01 12:05:41,203.0.113.45,10.0.0.30,USA,India,Malware Download,Critical,TCP,60432,443,210000,98000,320,22,240,Quarantined,Mitigated,1


In [ ]:
import sqlite3

# Create in-memory SQLite DB
conn = sqlite3.connect(":memory:")

# Load dataframe into SQL table
df.to_sql("cyber_threats", conn, index=False, if_exists="replace")


10

In [ ]:
pd.read_sql("PRAGMA table_info(cyber_threats);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,incident_id,TEXT,0,None,0
1,1,timestamp,TEXT,0,None,0
2,2,src_ip,TEXT,0,None,0
3,3,dest_ip,TEXT,0,None,0
4,4,src_country,TEXT,0,None,0
5,5,dest_country,TEXT,0,None,0
6,6,attack_type,TEXT,0,None,0
7,7,severity,TEXT,0,None,0
8,8,protocol,TEXT,0,None,0
9,9,src_port,INTEGER,0,None,0


In [ ]:
pd.read_sql("""
SELECT COUNT(*) AS total_records
FROM cyber_threats;
""", conn)


,total_records
0,10


In [ ]:
pd.read_sql("""
SELECT
    is_malicious,
    COUNT(*) AS count
FROM cyber_threats
GROUP BY is_malicious;
""", conn)


,is_malicious,count
0,0,3
1,1,7


In [ ]:
pd.read_sql("""
SELECT
    attack_type,
    COUNT(*) AS incidents
FROM cyber_threats
GROUP BY attack_type
ORDER BY incidents DESC;
""", conn)


,attack_type,incidents
0,Normal Traffic,3
1,SQL Injection,1
2,Port Scan,1
3,Phishing,1
4,Malware Download,1
5,Insider Threat,1
6,DDoS,1
7,Brute Force,1


In [ ]:
pd.read_sql("""
SELECT
    attack_type,
    COUNT(*) AS malicious_count
FROM cyber_threats
WHERE is_malicious = 1
GROUP BY attack_type
ORDER BY malicious_count DESC
LIMIT 5;
""", conn)


,attack_type,malicious_count
0,SQL Injection,1
1,Port Scan,1
2,Phishing,1
3,Malware Download,1
4,Insider Threat,1


In [ ]:
pd.read_sql("""
SELECT
    protocol,
    ROUND(AVG(is_malicious) * 100, 2) AS malicious_percentage
FROM cyber_threats
GROUP BY protocol
ORDER BY malicious_percentage DESC;
""", conn)


,protocol,malicious_percentage
0,TCP,87.5
1,UDP,0.0


In [ ]:
pd.read_sql("""
SELECT
    src_country,
    COUNT(*) AS total_attacks,
    SUM(is_malicious) AS malicious_attacks
FROM cyber_threats
GROUP BY src_country
ORDER BY malicious_attacks DESC;
""", conn)


,src_country,total_attacks,malicious_attacks
0,India,4,2
1,Ukraine,1,1
2,USA,2,1
3,UK,1,1
4,Russia,1,1
5,Germany,1,1


In [ ]:
pd.read_sql("""
SELECT *
FROM cyber_threats
WHERE bytes_sent > 5000
AND is_malicious = 1
LIMIT 10;
""", conn)


,incident_id,timestamp,src_ip,dest_ip,src_country,dest_country,attack_type,severity,protocol,src_port,dest_port,bytes_sent,bytes_received,packets,duration_sec,response_time_ms,action_taken,attack_status,is_malicious
0,INC001,2025-01-01 10:12:45,185.220.101.5,10.0.0.12,Russia,India,DDoS,High,TCP,44321,80,950000,120000,1200,35,180,Blocked IP,Mitigated,1
1,INC002,2025-01-01 10:25:10,192.168.1.23,10.0.0.15,India,India,Port Scan,Medium,TCP,51514,22,12000,5000,150,12,90,Logged,Detected,1
2,INC003,2025-01-01 11:02:33,45.83.64.19,10.0.0.20,Germany,India,Brute Force,High,TCP,39822,22,48000,14000,420,18,110,Account Locked,Mitigated,1
3,INC005,2025-01-01 12:05:41,203.0.113.45,10.0.0.30,USA,India,Malware Download,Critical,TCP,60432,443,210000,98000,320,22,240,Quarantined,Mitigated,1
4,INC006,2025-01-01 12:48:10,10.1.1.14,10.0.0.40,India,India,Insider Threat,High,TCP,49210,445,650000,340000,680,55,150,User Disabled,Investigating,1
5,INC008,2025-01-01 13:45:59,91.214.124.7,10.0.0.60,Ukraine,India,SQL Injection,Critical,TCP,38214,3306,160000,42000,210,16,210,Blocked IP,Mitigated,1
6,INC009,2025-01-01 14:20:03,192.0.2.19,10.0.0.70,UK,India,Phishing,Medium,TCP,49821,25,24000,12000,95,10,75,Email Filtered,Detected,1
